# Support Vector Classification

This is a component that trains a Support Vector Classification model using [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load datasets, save models and other artifacts.
- how to declare parameters and use them to build reusable components.

## Declare parameters and model hyperparameters
Components may declare (and use) these default parameters:
- dataset
- target

Use these parameters to load/save datasets, models, metrics, and figures with the help of [PlatIAgro SDK](https://platiagro.github.io/sdk/). <br />
You may also declare custom parameters to set when running an experiment.

Select the hyperparameters and their respective values to be used when training the model:
- C
- kernel
- degree
- gamma
- probability
- max_iter

These parameters are just a few offered by the model class, you may also use another existing parameter. <br />
Check the [model parameters](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) for more information.

In [ ]:
# parameters
dataset = "iris" #@param {type:"string"}
target = "Species" #@param {type:"string"}

# hyperparameters
C = 1.0 #@param {type:"number", label:"Regularização", description:"A força da regularização é inversamente proporcional a C. Deve ser positivo. Penalidade é l2²"}
kernel = "rbf" #@param ["linear", "poly", "rbf", "sigmoid", "precomputed"] {type:"string", label:"Kernel", description:"Especifica o tipo de kernel a ser usado no algoritmo"}
degree =  3 #@param {type:"integer", label:"Grau", description:"Grau da função polinomial do kernel ('poly'). Ignorado por outros kernels"}
gamma = "auto" #@param ["scale", "auto"] {type: "string", label:"Gama", description:"Coeficiente de kernel para 'rbf', 'poly' e 'sigmoid'"} 
probability = True #@param {type: "boolean", label:"Probabilidade", description:"Se é necessário ativar estimativas de probabilidade.  Deve ser ativado antes da chamada fit() do modelo, reduzirá a velocidade desse método, pois ele usa internamente o 5-fold-cross-validation, e predict_proba pode ser inconsistente com a chamada predict()"}
max_iter = -1 #@param {type:"integer", label:"Iteração", description:"Limite fixo nas iterações no solver, ou -1 sem limite"}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Encode target labels

The target labels are converted to ordinal integers with value between 0 and n_classes-1.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

## Split dataset into train/test splits

Training Dataset: the sample of data used to fit the model.

Test Dataset: the sample of data used to provide an unbiased evaluation of a model fit on the training dataset.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,  train_size=0.7)

## Fit a model using sklearn.svm.SVC

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from platiagro.featuretypes import NUMERICAL
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

numerical_indexes = (featuretypes == NUMERICAL)

pipeline = Pipeline(steps=[
    ('handle missing values', ColumnTransformer(
    [('imputer_mean', SimpleImputer(strategy='mean'), numerical_indexes),
     ('imputer_mode', SimpleImputer(strategy='most_frequent'), ~numerical_indexes)],
    remainder='drop')),
    ('handle categorical features', ColumnTransformer(
    [('feature_encoder', OrdinalEncoder(), ~numerical_indexes)],
    remainder='passthrough')),
    ('estimator', SVC(C=C,
                      kernel=kernel,
                      degree=degree,
                      gamma=gamma,
                      probability=probability,
                      max_iter=max_iter)),
])

pipeline.fit(X_train, y_train)

## Measure the performance
The [**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix) is a performance measurement for machine learning classification.<br>
It is extremely useful for measuring [Accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_binary_classification), [Recall, Precision, and F-measure](https://en.wikipedia.org/wiki/Precision_and_recall).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

# uses the model to make predictions on the Test Dataset
y_pred = pipeline.predict(X_test)

# computes confusion matrix
labels = np.unique(y)
data = confusion_matrix(y_test, y_pred, labels=labels)

# puts matrix in pandas.DataFrame for better format
labels = label_encoder.inverse_transform(labels)
confusion_matrix = pd.DataFrame(data, columns=labels, index=labels)

## Save metrics

Record the metrics used to evaluate the model.<br>
It's a good way to document the experiments, and also help to avoid running the same experiment twice. 

In [ ]:
from platiagro import save_metrics

save_metrics(confusion_matrix=confusion_matrix)
pipeline.fit(X, y)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline,
           label_encoder=label_encoder,
           columns=columns)